In [6]:
# Load libraries
library(readxl)
library(pROC)
library(dplyr)
library(janitor)

# ✅ STEP 1: Load dataset
data_path <- "/kaggle/input/raw-data-jaksa/Raw_data_JB_Ozlem.xlsx"
clean_data <- read_excel(data_path) %>%
  janitor::clean_names()  # normalize column names

# Define biomarkers and days
biomarkers <- c("presepsin", "ykl_40", "crp", "pct")
days <- c("d1", "d3", "d5", "d7")

# Initialize result storage
cutoff_summary <- data.frame()
uni_models <- list()
multi_models <- list()

# Loop over each day
for (day in days) {
  cat("\n### ---- DAY:", toupper(day), "---- ###\n")
  
  for (bm in biomarkers) {
    # Adjust suffixes
    unit_suffix <- case_when(
      bm == "crp" ~ "_mg_l",
      bm == "pct" ~ "_mcg_l",
      TRUE ~ "_ng_m_l"
    )
    col_name <- paste0(bm, "_", day, unit_suffix)
    if (!(col_name %in% colnames(clean_data))) next
    
    # Prepare temp data frame
    temp_df <- clean_data %>%
      select(sepsis = sepsis_yes_no, value = all_of(col_name)) %>%
      mutate(value = as.numeric(value)) %>%
      filter(!is.na(value), !is.na(sepsis))
    
    if (nrow(temp_df) < 10 || length(unique(temp_df$sepsis)) != 2) next
    
    # ROC + Youden
    roc_obj <- roc(temp_df$sepsis, temp_df$value, quiet = TRUE)
    youden <- roc_obj$sensitivities + roc_obj$specificities - 1
    best_idx <- which.max(youden)
    
    cutoff <- roc_obj$thresholds[best_idx]
    auc <- auc(roc_obj)
    sens <- roc_obj$sensitivities[best_idx]
    spec <- roc_obj$specificities[best_idx]
    
    # Store results
    cutoff_summary <- rbind(cutoff_summary, data.frame(
      Day = day,
      Biomarker = bm,
      Cutoff = cutoff,
      AUC = auc,
      Sensitivity = sens,
      Specificity = spec
    ))
    
    # Create binary variable
    bin_var <- paste0(bm, "_bin_", day)
    clean_data[[bin_var]] <- clean_data[[col_name]] > cutoff
    
    # Univariate logistic regression
    uni_formula <- reformulate(bin_var, response = "sepsis_yes_no")
    uni_model <- glm(uni_formula, data = clean_data, family = binomial)
    uni_models[[paste0(bm, "_", day)]] <- summary(uni_model)
    
    # Console logging
    cat("  ➤", bm, "Cutoff =", round(cutoff, 4), "| AUC =", round(auc, 3), "\n")
    
    # Log for PCT (all days)
    if (bm == "pct") {
      if (day == "d1") {
        if (cutoff >= 0.15 && cutoff <= 2.15) {
          cat("    ✅ PCT Day 1 cutoff within expected range (0.15–2.15 µg/L):", round(cutoff, 4), "\n")
        } else {
          cat("    ⚠️ WARNING: PCT Day 1 cutoff outside expected range (0.15–2.15 µg/L):", round(cutoff, 4), "\n")
        }
      } else {
        cat("    ℹ️ Note: PCT", toupper(day), "cutoff logged (no predefined range):", round(cutoff, 4), "\n")
      }
    }
  }
  
  # Multivariate model: Presepsin + PCT
  pres_bin <- paste0("presepsin_bin_", day)
  pct_bin <- paste0("pct_bin_", day)
  
  if (all(c(pres_bin, pct_bin) %in% colnames(clean_data))) {
    multi_formula <- reformulate(c(pres_bin, pct_bin), response = "sepsis_yes_no")
    multi_model <- glm(multi_formula, data = clean_data, family = binomial)
    multi_models[[paste0("multi_", day)]] <- summary(multi_model)
    
    cat("  ✅ Multivariate model run for", day, "(Presepsin + PCT)\n")
  }
}

# Show final cutoff summary table
print(cutoff_summary)



### ---- DAY: D1 ---- ###
  ➤ presepsin Cutoff = 4.788 | AUC = 0.792 
  ➤ ykl_40 Cutoff = 105.55 | AUC = 0.66 
  ➤ crp Cutoff = 56.05 | AUC = 0.593 
  ➤ pct Cutoff = 2.135 | AUC = 0.749 
    ✅ PCT Day 1 cutoff within expected range (0.15–2.15 µg/L): 2.135 
  ✅ Multivariate model run for d1 (Presepsin + PCT)

### ---- DAY: D3 ---- ###
  ➤ presepsin Cutoff = 4.0905 | AUC = 0.885 
  ➤ ykl_40 Cutoff = 108.4 | AUC = 0.696 
  ➤ crp Cutoff = 29.85 | AUC = 0.597 
  ➤ pct Cutoff = 0.545 | AUC = 0.882 
    ℹ️ Note: PCT D3 cutoff logged (no predefined range): 0.545 
  ✅ Multivariate model run for d3 (Presepsin + PCT)

### ---- DAY: D5 ---- ###
  ➤ presepsin Cutoff = 3.598 | AUC = 0.823 
  ➤ ykl_40 Cutoff = 80.65 | AUC = 0.558 
  ➤ crp Cutoff = 143.35 | AUC = 0.591 
  ➤ pct Cutoff = 0.365 | AUC = 0.875 
    ℹ️ Note: PCT D5 cutoff logged (no predefined range): 0.365 
  ✅ Multivariate model run for d5 (Presepsin + PCT)

### ---- DAY: D7 ---- ###
  ➤ presepsin Cutoff = 1.705 | AUC = 0.753 
  ➤ ykl_4

### Step 6B & 6C Summary: Diagnostic Cutoffs and Logistic Models (Day 1–7)

---

### ROC & Youden-Based Cutoffs

| **Day** | **Biomarker** | **Cutoff** | **AUC** | **Sensitivity** | **Specificity** |
|--------|----------------|------------|---------|------------------|------------------|
| D1     | Presepsin      | 4.788      | 0.792   | 0.778            | 0.750            |
| D1     | YKL-40         | 105.55     | 0.660   | 0.667            | 0.729            |
| D1     | CRP            | 56.05      | 0.593   | 0.611            | 0.688            |
| D1     | PCT            | 2.135      | 0.749   | 0.500            | 0.979            |
| D3     | Presepsin      | 4.0905     | 0.885   | 0.944            | 0.708            |
| D3     | YKL-40         | 108.4      | 0.696   | 0.611            | 0.771            |
| D3     | CRP            | 29.85      | 0.597   | 1.000            | 0.250            |
| D3     | PCT            | 0.545      | 0.882   | 0.833            | 0.875            |
| D5     | Presepsin      | 3.598      | 0.823   | 0.722            | 0.792            |
| D5     | YKL-40         | 80.65      | 0.558   | 0.500            | 0.688            |
| D5     | CRP            | 143.35     | 0.591   | 0.222            | 1.000            |
| D5     | PCT            | 0.365      | 0.875   | 0.778            | 0.854            |
| D7     | Presepsin      | 1.705      | 0.753   | 0.889            | 0.500            |
| D7     | YKL-40         | 56.75      | 0.645   | 0.722            | 0.563            |
| D7     | CRP            | 65.80      | 0.690   | 0.500            | 0.833            |
| D7     | PCT            | 0.140      | 0.888   | 0.889            | 0.750            |

---

### Cutoff Validation Notes

-  **Day 1 PCT cutoff (2.135 µg/L)** is **within the clinically expected range (0.15–2.15 µg/L)**.
-  **Days 3, 5, 7 PCT cutoffs** were data-driven and fall outside this range. No constraints were predefined for these days.

---

### Logistic Modeling Overview

#### **Univariate Analysis (Step 6B)**  
ROC-derived binary predictors were used in logistic regression.  
**Presepsin** and **PCT** consistently showed the strongest diagnostic associations.

#### **Multivariate Analysis (Step 6C)**  
Multivariate logistic regression models were built using **Presepsin + PCT** for each day:

| **Day** | **Model**              | **Notes**                                      |
|--------|------------------------|-----------------------------------------------|
| D1     | ✅ Presepsin + PCT      | PCT cutoff validated and both predictors used |
| D3     | ✅ Presepsin + PCT      | Strong model with high AUCs                   |
| D5     | ✅ Presepsin + PCT      | PCT AUC: 0.875, Presepsin AUC: 0.823          |
| D7     | ✅ Presepsin + PCT      | Highest PCT AUC (0.888) across all days       |

---

